In [ ]:
#Para versão recente do python
!pip install tensorflow==2.8.0 tensorflow-gpu==2.8.0 opencv-python mediapipe sklearn matplotlib

In [ ]:
!pip uninstall protobuf

In [ ]:
# Para versões antigas - https://www.python.org/ftp/python/3.7.4/
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python==4.5.2.54 mediapipe==0.8.5 sklearn matplotlib

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
#MediaPipe solutions - reconhecimento e desenho dos pontos na mão
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [3]:
def mediapipe_detection(image, model):
    """ 
    Funcao com objetivo de aplicar a previsao de reconhecimento na imagem desejada 
    Recebe imagem (frame da webcam) e um modelo que sera responsavel pela previsão
    Retorna o frame utilizado e a previsao feita pelo modelo
    A conversao da imagem se faz necessaria para ser tratada pelo modelo (BGR -> RGB)
    A mudanca na propriedade de leitura tem como objetivo salvar memoria
    """
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [4]:
def draw_landmarks(image, results):
    """
    Funcao que recebe uma imagem (frame) junto com as previsoes feitas pelo modelo e aplica
    sob a imagem o desenho dos pontos necessarios (nao ha motivo para devolver a imagem pois ela
    ja e alterada diretamente).
    """
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             )
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             )
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             )
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

In [5]:
def extract_keypoints(results):
    '''É preciso guardar todos os pontos lidos pelo modelo em um numpy array
    Este array pose, por exemplo, armazena 33 numpy arrays que guardam, cada um, um ponto
    lido pelo modelo (com 3 coordenadas e visibility). Representando um único frame
    Depois o array é tratado de forma a transformá-lo num único array com todos os pontos.
    É retornado um array que junta todos os pontos lidos pela câmera.'''
    if results.pose_landmarks:      #Verifica se apareceu na câmera uma pessoa (array não vazio)
        pose = []
        for res in results.pose_landmarks.landmark:
            test = np.array([res.x, res.y, res.z, res.visibility])
            pose.append(test)

        np.array(pose).flatten()    #Trata os dados de forma a deixá-los todos em um único array de 33 vezes 4 = 132
    else:                           #Caso esteja vazio, cria um numpy array com zeros
        pose = np.zeros(132) 
        
    if results.left_hand_landmarks:  #Verifica se apareceu na câmera a mão esquerda (array não vazio)
        lefthand = []
        for res in results.left_hand_landmarks.landmark:
            test = np.array([res.x, res.y, res.z])
            lefthand.append(test)

        np.array(lefthand).flatten() #Trata os dados de forma a deixá-los todos em um único array de 21 vezes 3 = 63
    else:                            #Caso esteja vazio, cria um numpy array com zeros
        lefthand = np.zeros(63) 

    if results.right_hand_landmarks:  #Verifica se apareceu na câmera a mão direita (array não vazio)
        righthand = []
        for res in results.right_hand_landmarks.landmark:
            test = np.array([res.x, res.y, res.z])
            righthand.append(test)

        np.array(righthand).flatten() #Trata os dados de forma a deixá-los todos em um único array de 21 vezes 3 = 63
    else:                             #Caso esteja vazio, cria um numpy array com zeros
        righthand = np.zeros(63) 

    if results.face_landmarks:     #Verifica se apareceu na câmera a mão direita (array não vazio)
        face = []
        for res in results.face_landmarks.landmark:
            test = np.array([res.x, res.y, res.z])
            face.append(test)

        np.array(face).flatten()    #Trata os dados de forma a deixá-los todos em um único array de 468 vezes 3 = 1404
    else:                           #Caso esteja vazio, cria um numpy array com zeros
        face = np.zeros(1404) 
    
    return np.concatenate([pose, face, lefthand, righthand]) #Concatenação de todos pontos

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [7]:
# Caminho que guarda o numpy array com os pontos extraidos
DATA_PATH = os.path.join('MP_Data')

# Sinais que serão detectados
actions = np.array(['OLA', 'OBRIGADO', 'PRAZER EM CONHECER VOCE', 'PROFESSORA', 'QUAL O SEU NOME', 'TUDO BEM', 'EU SOU SURDO', 'ESTOU APRENDENDO', 'LIBRAS', 'POR ASSISTIR'])

# Representa a quantidade de sequências de frames que tem os dados
no_sequences = 100

# Representa a quantidade de frames que cada sequência possui
sequence_lenght = 30

In [8]:
# Cria pastas para cada sinal definido
# Cada pasta tem 30 pastas representando os videos modelo do sinal
# Cada vídeo contém 30 frames e cada frame 1662 pontos extraidos
for action in actions:
    for sequence in range(no_sequences):
        try: # Cria pastas e subpastas
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except: # Caso já exista passa para próxima pasta
            pass

In [ ]:
#Acessa a webcam - valor (0) representa o hardware
cap = cv2.VideoCapture(0)

#Define o modelo utilizado
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # Extrai os frames de cada video para cada ação
    for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_lenght):

                #Seleciona o frame atual da webcam
                ret, frame = cap.read()

                #Processa previsoes
                image, results = mediapipe_detection(frame, holistic)
                #print(results)

                #Desenha os pontos
                draw_landmarks(image, results)
                
                # Interface e espera ao começar gravações
                if frame_num == 0:
                    
                    #Textos para indicar inicio de gravação
                    cv2.putText(image, "INICIANDO GRAVAÇÃO DE SINAL", 
                    (120, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv2.LINE_AA)
                    
                    cv2.putText(image, "Gravando para {} Video: {}".format(action, sequence), 
                    (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    
                    # Tempo de espera entre um vídeo e outro
                    cv2.waitKey(2000)
                else:
                    cv2.putText(image, "Gravando para {} Video: {} Frame: {}".format(action, sequence, frame_num), 
                    (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                
                # Extrai os pontos de um frame e salva como um arquivo .npy (numpy array)
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)
                
                #Faz o display do frame
                if ret == True: 
                    cv2.imshow('Webcam', image)

                    #Encerra a webcam 
                    if cv2.waitKey(10) & 0xFF == ord('q'):
                        break
                else:
                    break
    cap.release()
    cv2.destroyAllWindows()

In [22]:
# Imports que serão importante para treinar o modelo e nomear os dados
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [23]:
label_map = {label:num for num, label in enumerate(actions)}
sequences, labels = [], []

# Carrega e adiciona todos os pontos de uma sequência no array sequences
# Enquanto o array labels coloca em ordem o valor de determinada ação com certa sequencia
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_lenght):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence),'{}.npy'.format(frame_num+1)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])
    
    matrix_labels = to_categorical(labels).astype(int)
    X = np.array(sequences)
    
    # Com os dados já definidos é possível separar uma quantidade para teste e treinamento do modelo
    x_train, x_test, y_train, y_test = train_test_split(X, matrix_labels, test_size=0.05)

In [24]:
# Imports para construção da rede neural
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [25]:
# Definição do caminho para registro de logs da rede
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [45]:
del model

In [26]:
# Setup da rede neural
model = Sequential()

# Cada função add adiciona uma layer para a rede
# O parâmetro return_sequences é importante para passar para próxima layer de forma ordenada
# O parâmetro activation define a função que será utilizada para ativar os neurônios
#  O parâmetro input_shape = 30 frames cada um com 1662 pontos reconhecidos
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662))) 
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))

# Última camada possui a quantidades de pontos definido pela quantidade de ações
# A função softmax entrega um resultado com a probabilidade de cada ação (será selecionado a com maior probabilidade)
model.add(Dense(actions.shape[0], activation='softmax'))

In [27]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [31]:
model.fit(x_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
30/30 [==============================] - 3s 85ms/step - loss: 0.0281 - categorical_accuracy: 0.9884
Epoch 2/2000
30/30 [==============================] - 2s 51ms/step - loss: 0.0659 - categorical_accuracy: 0.9737
Epoch 3/2000
30/30 [==============================] - 2s 52ms/step - loss: 0.1191 - categorical_accuracy: 0.9653
Epoch 4/2000
30/30 [==============================] - 2s 51ms/step - loss: 0.1648 - categorical_accuracy: 0.9516
Epoch 5/2000
30/30 [==============================] - 2s 53ms/step - loss: 0.0439 - categorical_accuracy: 0.9811
Epoch 6/2000
20/30 [===================>..........] - ETA: 0s - loss: 0.0409 - categorical_accuracy: 0.9844

KeyboardInterrupt: 

In [29]:
# Mostra a performance do modelo
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 30, 64)            442112    
_________________________________________________________________
lstm_4 (LSTM)                (None, 30, 128)           98816     
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_3 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_5 (Dense)              (None, 10)                330       
Total params: 596,906
Trainable params: 596,906
Non-trainable params: 0
________________________________________________

In [29]:
# Retorna uma lista com valores de probabilidade de cada ação como definidos no modelo
res = model.predict(x_test)

In [59]:
# Salva os weights usados pelo modelo
model.save('action.h5')
#model.load_weights(action.h5)
#del model

In [15]:
model.load_weights('action.h5')

In [32]:
# Teste de métricas para aprovação do modelo
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [33]:
yhat = model.predict(x_test)
#yhat = model.predict(x_train)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

# Retorna uma matriz bidimensional que representa se houve falsos positivos ou negativos
# Quanto maior a quantidade do valor de um lado da matriz melhor o modelo
multilabel_confusion_matrix(ytrue, yhat)

array([[[44,  0],
        [ 0,  6]],

       [[45,  0],
        [ 0,  5]],

       [[42,  0],
        [ 0,  8]],

       [[44,  0],
        [ 0,  6]],

       [[41,  0],
        [ 0,  9]],

       [[46,  0],
        [ 0,  4]],

       [[47,  0],
        [ 0,  3]],

       [[48,  0],
        [ 0,  2]],

       [[44,  0],
        [ 0,  6]],

       [[49,  0],
        [ 0,  1]]], dtype=int64)

In [34]:
# Retorna a precisão do modelo
accuracy_score(ytrue, yhat)

1.0

In [37]:
colors = [(245,117,16), (117,245,16), (16,117,245), (16,117,245), (16,117,245), (16,117,245), (16,117,245), (16,117,245), (16,117,245), (16,117,245)]  #Array que guarda cores
def prob_viz(res, actions, input_frame, colors):
    """ Função que mostra na tela a probabilidade de cada um dos sinais, em tempo real.
    Recebe os resultados com as probabilidades, os sinais, um frame e as cores a serem utilizadas.
    Retorna o mesmo frame com as palavras e suas respectivas probabilidades.
    """
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):   #Para cada sinal cria os textos e probabilidades
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num] + str(round(res[num],2)), (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255,255,255), 1, cv2.LINE_AA)
        
    return output_frame

In [ ]:
sequence = []      # Guarda os frames
sentence = []      # Guarda quais foram as previsões feitas pelo modelo para formar frases 
predictions = []   # Guarda as prediçoes feita para evitar problemas na transição de sinais
threshold = 0.7
#Acessa a webcam - valor (0) representa o hardware
cap = cv2.VideoCapture(0)

#Define o modelo utilizado
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        #Seleciona o frame atual da webcam
        ret, frame = cap.read()

        #Processa previsoes
        image, results = mediapipe_detection(frame, holistic)
        
        #Desenha os pontos
        draw_landmarks(image, results)
        
        #Previsoes
        #Pega os pontos que estao aparecendo na camera e inserem em um array
        #Cada sequencia é feita com os 30 ultimos frames que depois será passada para o modelo
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        #Verifica se há alguma mão no frame, para só então começar a previsão
        rh_onscreen = (sequence[-1][-63:]!=np.zeros(63)).all()
        lh_onscreen = (sequence[-1][-126:-63]!=np.zeros(63)).all()
        
        #if len(sequences)==30:
        if len(sequence) == 30 and (rh_onscreen or lh_onscreen):
            #expand_dims permite passar uma sequencia por vez
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            #print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            #Visualização
            #Checa se os ultimos 10 frames são do mesmo sinal, para evitar problemas na transição de sinais
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                predictions = []
                #Compara se a probabilidade de certo sinal é maior que o minimo para mostrar
                if res[np.argmax(res)] > threshold: #seria possivel fazer a media?
                    #tentativa de zerar a sequencia apos previsão correta - ajuda?
                    if len(sentence) > 0:
                        # Verifica se o sinal já não está incluido na frase
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            #Caso a frase fique muito grande, guarda apenas os ultimos valores
            if len(sentence) > 5:
                sentence = sentence[-5:]
        
            #Renderiza as probabilidades
            image = prob_viz(res, actions, image, colors)
        
        #Renderiza predições e a frase
        cv2.rectangle(image, (0,0), (640,40), (245,117,16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 1, cv2.LINE_AA)
        
        
        #Faz o display do frame
        if ret == True: 
            cv2.imshow('Webcam', image)

            #Encerra a webcam 
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
            
        else:
            break
    cap.release()
    cv2.destroyAllWindows()

In [47]:
cap.release()
cv2.destroyAllWindows()

In [32]:
sequence[-1][-126:-63].shape

(63,)

In [28]:
testearray = np.zeros(1662)

In [47]:
sequence2 = []
for seq in sequence:
    sequence2.append(np.zeros(1662))

In [51]:
np.array(sequence2).shape

(30, 1662)

In [50]:
np.array(sequence).shape

(30, 1662)

In [57]:
res = model.predict(np.expand_dims(sequence, axis=0))[0]

array([1.8321335e-01, 1.5490651e-04, 8.1663173e-01], dtype=float32)